In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from count import *

In [ ]:
def score_diff_hist(data):
    hist = {}
    for (a, b), c in data.items():
        hist[a - b] = hist.get(a - b, 0) + c
    xx = np.arange(min(hist), max(hist) + 1)
    return xx, np.array([hist.get(x, 0) for x in xx])

In [ ]:
def hoshitori(data_set, offset=0):
    hoshi_a = {}
    hoshi_b = {}
    for (name_a, name_b), data in data_set.items():
        if name_a not in hoshi_a:
            hoshi_a[name_a] = [0, 0, 0]
        if name_b not in hoshi_b:
            hoshi_b[name_b] = [0, 0, 0]
        for (sa, sb), c in data.items():
            score = sa - (sb + offset)
            if score == 0:
                hoshi_a[name_a][2] += c
                hoshi_b[name_b][2] += c
            elif score > 0:
                hoshi_a[name_a][0] += c
                hoshi_b[name_b][1] += c
            else:
                hoshi_a[name_a][1] += c
                hoshi_b[name_b][0] += c
    return hoshi_a, hoshi_b

def order_hoshi_by_win_rate(hoshi):
    for name in sorted(hoshi, key=lambda n: hoshi[n][0] / sum(hoshi[n]), reverse=True):
        h = hoshi[name]
        print(name, h, h[0] / sum(h))

In [ ]:
import texttable

def to_table(data_set):
    names = list(set([name for name, _ in data_set]) | set([name for _, name in data_set]))
    names.sort()

    table = texttable.Texttable(max_width=500)
    table.set_cols_align(['l'] * (len(names) + 2))
    table.add_row([''] + names + [''])
    for name1 in names:
        row = [name1]
        total_win = total_lose = total_draw = 0
        for name2 in names:
            win = lose = draw = 0
            for (sa, sb), c in data_set.get((name1, name2), {}).items():
                if sa > sb:
                    win += c
                elif sa < sb:
                    lose += c
                else:
                    draw += c
            total = win + lose + draw
            if total:
                row.append(f'{win}-{lose}-{draw}\n({100 * win / total:.1f})')
            else:
                row.append('')
            total_win += win
            total_lose += lose
            total_draw += draw
        total = total_win + total_lose + total_draw
        if total:
            row.append(f'{total_win}-{total_lose}-{total_draw}\n({100 * total_win / total:.1f})')
        else:
            row.append('')
        table.add_row(row)
    
    row = ['']
    for name1 in names:
        win = lose = draw = 0
        for name2 in names:
            for (sa, sb), c in data_set.get((name2, name1), {}).items():
                if sa < sb:
                    win += c
                elif sa > sb:
                    lose += c
                else:
                    draw += c
        total = win + lose + draw
        if total:
            row.append(f'{win}-{lose}-{draw}\n({100 * win / total:.1f})')
        else:
            row.append('')
    row.append('')
    table.add_row(row)
    return table.draw()

In [ ]:
data = {}
for f in Path('.').glob('*.txt'):
    count(f, data)

hoshi_a, hoshi_b = hoshitori(data, offset=0)

In [ ]:
order_hoshi_by_win_rate(hoshi_a)

In [ ]:
order_hoshi_by_win_rate(hoshi_b)

In [ ]:
print(to_table(data))